In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from data import *
from window import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def backtest_strategy(data):
    """ Simulate trading based on provided buy and sell signals, including tracking entry and exit prices. """
    capital = 10000
    shares = 0

    # Create portfolio dataframe 
    portfolio = pd.DataFrame(index=data.index, columns=['Portfolio Value', 'Entry Price', 'Position Type'])
    entry_price = None
    prev_signal = None
    in_position = False

    for row_index in range(0, len(data)): 
        signal = generate_signals(data, row_index)
        current_price = data.iloc[row_index]['Close']
    
        if (signal == 'Buy' and not in_position) and (prev_signal != 'Buy' and capital > 0):  # Check if signal is 'Buy' and different from previous
            shares_to_buy = capital / current_price
            shares += shares_to_buy
            capital -= shares_to_buy * current_price
            entry_price = current_price

            # Enter a new position
            in_position = True 
            portfolio.loc[data.index[row_index], 'Position Type'] = 'Long'

        if (signal == 'Sell' and prev_signal != 'Sell' and shares > 0):
            capital += shares * current_price
            shares = 0
            # Exit the position
            entry_price = None
            in_position = False
            portfolio.loc[data.index[row_index], 'Position Type'] = 'None'

        # Calculate portfolio value
        portfolio_value = capital + (shares * current_price)
        portfolio.loc[data.index[row_index], 'Entry Price'] = entry_price
        portfolio.loc[data.index[row_index], 'Portfolio Value'] = portfolio_value
        
        prev_signal = signal  # Update previous signal for next iteration

    return portfolio.fillna(0)

def calculate_performance(portfolio):
    """ Calculate the percent change in portfolio value from the first day to the last. """
    first_day_value = portfolio.iloc[0]['Portfolio Value']
    last_day_value = portfolio.iloc[-1]['Portfolio Value']
    percent_change = ((last_day_value - first_day_value) / first_day_value) * 100
    return percent_change

In [ ]:
# Set start date as 60 days before today and end date as today
start_date = (datetime.today() - timedelta(days=59)).strftime('%Y-%m-%d')
end_date = datetime.today().strftime('%Y-%m-%d')

# Running the backtest
tickers = ['SPY', 'NDAQ','JPM', 'C', 'BAC', 'PANW', 'SHOP', 'NET', 'TSM', 'CRM', 'ON', 'SONY', 'CRWD', 'AMAT', 'AMZN', 'AAPL']

for ticker in tickers: 
    data = fetch_data(ticker, start_date, end_date)
    portfolio = backtest_strategy(data)
    performance = calculate_performance(portfolio)
    print(f"Performance of {ticker} is {performance:.2f}%")

# Display results
print(portfolio)

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


ticker = 'LTC-USD'
start_date = (datetime.today() - timedelta(days=59)).strftime('%Y-%m-%d')
end_date = datetime.today().strftime('%Y-%m-%d')

data = fetch_data(ticker, start_date, end_date)

# Normalizing the stock data
stock_data = data['Close'] * 20
initial_stock_value = stock_data.iloc[0]
normalized_stock_data = (stock_data / initial_stock_value) * 10000

# Backtest the strategy
portfolio = backtest_strategy(data)

# Normalizing the portfolio data
initial_portfolio_value = portfolio['Portfolio Value'].iloc[0]
normalized_portfolio = (portfolio['Portfolio Value'] / initial_portfolio_value) * 10000

performance = calculate_performance(portfolio)
print(f"Performance of {ticker} is {performance:.2f}%")

# Plotting the data
plt.figure(figsize=(10, 6))
plt.plot(portfolio.index, normalized_portfolio, label='Portfolio')
plt.plot(stock_data.index, normalized_stock_data, label=f'{ticker} Price')
plt.title(f'Portfolio vs {ticker} Performance')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()